In [1]:
import numpy as np
import scipy.optimize
import scipy.interpolate as interp
import sys
from scipy import signal
from lmfit import minimize, Parameters
from multiprocessing import Process

from matplotlib.pyplot import *
import matplotlib.pyplot as plt

from matplotlib import rcParams
rcParams['font.family']='serif'
rcParams['mathtext.fontset'] = 'dejavuserif'

#Make np arrays print in scientific notation:
np.set_printoptions(formatter={'float': lambda x: format(x, '9.4E')})
#Make numbers display in exponential form:
%precision %e
end=1

# User input

In [2]:
scatter=[0.2,0.3] #scatter assumption in dex---Enter as many scatter assumptions as you want to evaluate here, separated by commas.
begbins_fromuser=8. #set the log M_* point at which the program will start evaluation
endbins_fromuser=11.4 # #set the log M_* point at which the program will end evaluation 

# Importing halo mass array from the Bolshoi simulation

In [3]:
MHALDATA = np.loadtxt ("halo_bolshoi_z0.00_D360.dat", unpack=True, usecols=1) #halo mass per 250/0.7 cubic megaparsecs

# Defining functions and bins
Hudson's $f_*(M_h)$ function<br>
Bins<br>
Baldry et. al. 2012 SMF<br>
SMF given by $f_*$<br>
Fitting function (error-weighted residuals)<br>

In [4]:
####################################################################################
#Hudson's f_*(M_h) function#########################################################
'''Defining Hudson's fstar function and  and starting the code off with some guesses
as to the parameters thereof.'''
def fstar(m_indp,m1,f1,beta,gamma,*params):
    '''Provide the option to push either a value 
    or the f1 function with associated parameters to the fstar function'''
    if callable(f1):
        f1val=f1(*params[:3])
    else:
        f1val=f1
    '''Provide the option to push either a value 
    or the m1 function with associated parameters to the fstar function'''
    if callable(m1):
        m1val=m1(*params[3:])
    else:
        m1val=m1
    
    fstar=2.*f1val*((m_indp/m1val)**-beta+(m_indp/m1val)**gamma)**-1.
    return fstar

'''Hudson allows f1, the stellar fraction at a given mass, and m1, a 
characteristic halo mass, to evolve with 
redshift, the method of which the following functions reproduce. In its current state, the program
does not actually use these functions'''
def f1(fpoint5,z,fz):
    f1=fpoint5+(z-0.5)*fz
    return f1
def m1(logmpoint5,z,mz):
    logm1=logmpoint5+(z-0.5)*mz
    m1=10.**logm1
    return m1

#Redshift
z=0

#f1 inputs
fpoint5_0=0.0414
fz_0=0.029
f1_0=0.022 #This is the value I'll use to override f1 instead of using the f1 function, when I choose to do so.

#m1 inputs
logmpoint5_0=11.04
mz_0=0.56
m1_0=10.**12. #This value will override m1 instead of using the m1 function when I choose to do so.
logm1_0=np.log10(m1_0)

#Other fstar inputs
beta_0=1.28
gamma_0=0.73

#Translating the initial guesses into a Parameters object that lmfit can use
p_chisqr=Parameters()
p_chisqr.add('logm1',value=logm1_0,vary=True,min=11.9,max=12.1)
p_chisqr.add('f1',value=f1_0,vary=True,min=0.017,max=0.025)
p_chisqr.add('beta',value=beta_0,vary=True,min=1.15,max=1.5)
p_chisqr.add('gamma',value=gamma_0,vary=True,min=0.5,max=1.)
####################################################################################

####################################################################################
#Creating bins######################################################################
LOGMSTARBINSIZE=0.05 #Bin size in units of dex
BEGBINS=begbins_fromuser
ENDBINS=endbins_fromuser
LOGMSTARBINS=np.arange(BEGBINS,ENDBINS+LOGMSTARBINSIZE,LOGMSTARBINSIZE) #M_* bins in dex (Add 1 to the num arg because the elements include the starting point.)
LOGMSTARMIDBINS=(LOGMSTARBINS[1:]+LOGMSTARBINS[:-1])/2
####################################################################################

####################################################################################
#Defining and calculating the SMF by Baldy et. al. 2012 and the associated errors###
def smf(m,m_ast,phi_ast1,phi_ast2,alpha1,alpha2):
    return np.exp(-m/m_ast)*(phi_ast1*(m/m_ast)**alpha1+phi_ast2*(m/m_ast)**alpha2)/m_ast*m

m_ast = 10.**10.66 #M* in solar mass
phi_ast1 = 3.96*10.**-3. #Mpc^-3
alpha1 = -0.35
phi_ast2 = 0.79*10.**-3. #Mpc^-3
alpha2 = -1.47

smftrue=smf(10.**LOGMSTARMIDBINS,m_ast,phi_ast1,phi_ast2,alpha1,alpha2)
logsmftrue=np.log10(smftrue)

Ntrue=smftrue*(250./0.7)**3.*LOGMSTARBINSIZE #Translating Baldry number density into Baldry number
frac_dsmftrue=1/np.sqrt(Ntrue) #Poisson fractional errors
dsmftrue=smftrue*frac_dsmftrue #calculating error on n from dn/n
dlogsmftrue=dsmftrue/(smftrue*np.log(10.)) #Propogating error to log(n)
####################################################################################

####################################################################################
#Defining a function to translate f_* into the corresponding SMF####################
'''Defining a function that will fill an array with the log number densities of log stellar-masses
given an array of log stellar masses'''

#Number density function:
def logn(p,logmstarmidbins,gaussar):
    if type(p) is Parameters:
        logm1=p['logm1'].value
        f1=p['f1'].value
        beta=p['beta'].value
        gamma=p['gamma'].value
    else:
        logm1,f1,beta,gamma=p
    
    m1=10.**logm1
    logmstarmean=np.log10(MHALDATA*fstar(MHALDATA,m1,f1,beta,gamma))
    logmstarscattered=logmstarmean+gaussar #adding the gaussian random deviate with specified standard deviation to the mean log(mstar(mhal)) array
        
    N, binsout, patches = hist(logmstarscattered,bins=LOGMSTARBINS,edgecolor='k',linewidth=0.2)
    plt.clf() #stops the histogram from displaying
     
    n=N/(250./0.7)**3./LOGMSTARBINSIZE #number of galaxies of a given mass per cubic Megaparsec per dex (diving by bin size to make the function independent of binsize, dividing by 0.7 is dividing by h)
    logn=np.where(n==0,-9.e9,np.log10(n)) #setting n to a small finite number where it would otherwise be -infinity
    
    global i
    i+=1
    if i%100==0: print 'iteration {0:0.0f}'.format(i)
   
    return logn
####################################################################################

####################################################################################
#Fitting function###################################################################
def weightedresidsfunction(p,logmstarmidbins,gaussar,Y):
    y=logn(p,logmstarmidbins,gaussar)
    resids=y-Y
    weightedresidsar=resids/dlogsmftrue
    return weightedresidsar
####################################################################################

# Optimization
Find the parameters of the Hudson fitting function so that, given a certain scatter, the resulting stellar mass function matches Baldry.

In [10]:
for s in scatter:
    gaussN=MHALDATA.size
    gaussar=np.random.normal(0,s,gaussN) #Calculating an array of Guassian random deviates with mean 0 and standard deviation specified by the user in dex

    i=0 #resetting the iteration counter
    
    print'\n{0:0.1f} DEX RUN'.format(s)
    
    hudsonfit=minimize(weightedresidsfunction,p_chisqr,args=(LOGMSTARMIDBINS,gaussar,logsmftrue),method='basinhopping') #Running the Chi^2 fit using basin hopping method
    lognfitar=logn(hudsonfit.params,LOGMSTARMIDBINS,gaussar) #calucating the SMF resulting from filling Hudson's f_* with the optimized parameters
    lognguessar=logn(p_chisqr,LOGMSTARMIDBINS,gaussar) #calucating the SMF resulting from filling Hudson's f_* with the guessed parameters
    chisqrval=np.sum(weightedresidsfunction(hudsonfit.params,LOGMSTARMIDBINS,gaussar,logsmftrue)**2.)
    redchisqrval=chisqrval/(LOGMSTARMIDBINS.size-4.)
    
    fig=plt.figure(figsize=(7,5))
    ax=fig.add_subplot(1,1,1)
    #ax.plot(LOGMSTARMIDBINS,lognguessar,'bd',label='0.2 dex mock guess',ms=5)
    ax.plot(LOGMSTARMIDBINS,lognfitar,'rd',label='{0:0.1f} dex mock fit'.format(s),ms=5)
    truecol=(0.,255./255.,0./255.)
    ax.errorbar(LOGMSTARMIDBINS,logsmftrue,dlogsmftrue,fmt='d',mfc=truecol,mec=truecol,capsize=4,ecolor='k', elinewidth=1,ms=4,label='Baldry')
    ax.grid()
    ax.legend(fontsize=11)
    ax.set_title('SMF resulting from {0:0.1f} dex mock versus Baldry'.format(s),fontsize=14)
    ax.set_ylabel('log(Mpc$^{-3}$ dex$^{-1}$)',fontsize=13)
    ax.set_xlabel('log M$_*/M_\odot$',fontsize=13)
    bbox_props=dict(boxstyle='square',fc='white') #Define properties of text box
    ax.text(8,-4,'log $M_1=$ {0:0.2f}'
            '\n$f_1=$ {1:0.4f}'
            '\n$\\beta=$ {2:0.3f}'
            '\n$\gamma=$ {3:0.3}'
            '\n$\\chi^2_r=$ {4:0.1f}'.format(hudsonfit.params['logm1'].value,
                                         hudsonfit.params['f1'].value,
                                         hudsonfit.params['beta'].value,
                                         hudsonfit.params['gamma'].value,
                                         redchisqrval),
           fontsize=14,bbox=bbox_props)
    ax.tick_params(axis='both',labelsize=11)
    plt.show()


0.3 DEX RUN


KeyboardInterrupt: 

# Using Bolshoi halo masses
The following cell populates $f_*$ with the halo masses that appear in the Bolshoi simulation.

In [ ]:
fstarofmhal=fstar(MHALDATA,m1_0,f1,beta_0,gamma_0,fpoint5_0,z,fz_0) #This would be the line to use to push the m1 function as opposed to a forced value for m1
#fstarofmhal=fstar(mhal,10.**12,f1,beta_0,gamma_0,fpoint5_0,z,fz_0,logmpoint5_0,z,mz_0) #Using this to force a value into m1 as opposed to using the m1 function with associated parameters

logfstarofmhal=np.log10(fstarofmhal) #Put f_*(M_h) in log

#Adding different scatters to log f_*
GAUSSN=MHALDATA.size
GAUSS1=np.random.normal(0,0.1,GAUSSN) #create an array of Gaussian random deviates with std dev 0.1
GAUSS2=np.random.normal(0,0.2,GAUSSN) #create an array of Gaussian random deviates with std dev 0.2
GAUSS3=np.random.normal(0,0.3,GAUSSN) #create an array of Gaussian random deviates with std dev 0.3
logfstarofmhalprime1=GAUSS1+logfstarofmhal 
logfstarofmhalprime2=GAUSS2+logfstarofmhal 
logfstarofmhalprime3=GAUSS3+logfstarofmhal 

#Defining M_* based on Bolshoi halo masses, as distinct from f_*(M_*/M_h)
logmstarofmhalprime1=logfstarofmhalprime1+np.log10(MHALDATA)
logmstarofmhalprime2=logfstarofmhalprime2+np.log10(MHALDATA)
logmstarofmhalprime3=logfstarofmhalprime3+np.log10(MHALDATA)

#Plotting Hudson's f_* using Bolshoi halo masses############################
plotbase=10 #The plots below will use a log scale for their y axes with base plotbase
xmin=10.**9.
xmax=max(MHALDATA)

fig=plt.figure(figsize=(8,6))
ax=fig.add_subplot(1,1,1)
ax.semilogx(MHALDATA,logfstarofmhal,'bo',markersize=0.3)
ax.set_xlim(xmin,xmax)
ax.set_title('Mean $f_*(M_h)$',fontsize=17)
ax.set_ylabel('log $f_*=M_*/M_h(\log\;h^{-1}_{70})$',fontsize=14)
ax.set_xlabel('$M_h(h_{70}^{-1}M_\odot)$',fontsize=14)
ax.tick_params(axis='both',labelsize=13)
ax.grid(True)
plt.show()
############################################################################

#Plotting Hudson's f_* using Bolshoi halo masses with scatter added#########
fig=plt.figure(figsize=(8,6))
ax=fig.add_subplot(1,1,1)
ax.semilogx(MHALDATA,logfstarofmhalprime3,'go',markersize=0.2)
ax.semilogx(MHALDATA,logfstarofmhalprime2,'bo',markersize=0.2)
ax.semilogx(MHALDATA,logfstarofmhalprime1,'ro',markersize=0.2)
ax.set_xlim(xmin,xmax)
ax.set_title('$f_*(M_h)$ including scatter',fontsize=17)
ax.set_ylabel('log $f_*=M_*/M_h(\log\;h^{-1}_{70})$',fontsize=14)
ax.set_xlabel('$M_h(h_{70}^{-1}M_\odot)$',fontsize=14)
ax.tick_params(axis='both',labelsize=13)
ax.grid(True)
plt.show()
############################################################################

#Showing just M_* based on Bolshoi masses, not M_*/M_h######################
fig=plt.figure(figsize=(8,6))
ax=fig.add_subplot(1,1,1)
ax.semilogx(MHALDATA,logmstarofmhalprime3,'go',markersize=0.2)
ax.semilogx(MHALDATA,logmstarofmhalprime2,'bo',markersize=0.2)
ax.semilogx(MHALDATA,logmstarofmhalprime1,'ro',markersize=0.2)
ax.set_xlim(xmin,xmax)
ax.set_title('$f_*(M_h)\cdot M_h$',fontsize=17)
ax.set_ylabel('log $M_*(\log\;M_\odot)$',fontsize=14)
ax.set_xlabel('$M_h(h_{70}^{-1}M_\odot)$',fontsize=14)
ax.tick_params(axis='both',labelsize=13)
ax.grid(True)
plt.show()
############################################################################

# Manually finding parameters based on $\chi^2_r$

In [ ]:
#Calculating the data###############################################################
logm1range=np.arange(11.8,14.,0.05)
redchisqroflogm1=np.array([])
for m in logm1range:
    p_redchisqr=[m,f1_0,beta_0,gamma_0]
    resids=logn2(p_redchisqr,LOGMSTARMIDBINS)-logsmftrue
    chisqroflogm1=np.sum((resids/dlogn2ar)**2.)
    redchisqroflogm1=np.append(redchisqroflogm1,chisqroflogm1/(LOGMSTARMIDBINS.size-2.))
    
f1range=np.arange(0.015,3.5e-2,0.1e-3)
redchisqroff1=np.array([])
for f in f1range:
    p_redchisqr=[logm1_0,f,beta_0,gamma_0]
    resids=logn2(p_redchisqr,LOGMSTARMIDBINS)-logsmftrue
    chisqroff1=np.sum((resids/dlogn2ar)**2.)
    redchisqroff1=np.append(redchisqroff1,chisqroff1/(LOGMSTARMIDBINS.size-2.))
    
betarange=np.arange(0.5,2.0,0.02)
redchisqrofbeta=np.array([])
for b in betarange:
    p_redchisqr=[logm1_0,f1_0,b,gamma_0]
    resids=logn2(p_redchisqr,LOGMSTARMIDBINS)-logsmftrue
    chisqrofbeta=np.sum((resids/dlogn2ar)**2.)
    redchisqrofbeta=np.append(redchisqrofbeta,chisqrofbeta/(LOGMSTARMIDBINS.size-2.))
    
gammarange=np.arange(0.1,0.9,0.03)
redchisqrofgamma=np.array([])
for g in gammarange:
    p_redchisqr=[logm1_0,f1_0,beta_0,g]
    resids=logn2(p_redchisqr,LOGMSTARMIDBINS)-logsmftrue
    chisqrofgamma=np.sum((resids/dlogn2ar)**2.)
    redchisqrofgamma=np.append(redchisqrofgamma,chisqrofgamma/(LOGMSTARMIDBINS.size-2.))
####################################################################################

#Plotting the results###############################################################
fig=plt.figure(figsize=(9,5))
plt.subplots_adjust(wspace = 0.37,hspace=0.7)
xlabelsize=12
ticklabelsize=11

ax1=fig.add_subplot(2,2,1)
ax1.semilogy(logm1range,redchisqroflogm1)
ax1.set_xlabel('log $M_1$ (log $M_\odot$)',fontsize=xlabelsize)
ax1.set_title('$\chi^2_r(M_1)$',fontsize=17)
ax1.tick_params(axis='both',labelsize=ticklabelsize)

ax2=fig.add_subplot(2,2,2)
ax2.semilogy(f1range,redchisqroff1)
ax2.set_xlabel('$f_1 (h_{70}^{-1})$',fontsize=xlabelsize)
ax2.set_title('$\chi^2_r(f_1)$',fontsize=17)
ax2.tick_params(axis='both',labelsize=ticklabelsize)

ax2=fig.add_subplot(2,2,3)
ax2.semilogy(betarange,redchisqrofbeta)
ax2.set_xlabel('$\\beta$',fontsize=xlabelsize)
ax2.set_title('$\chi^2_r(\\beta)$',fontsize=17)
ax2.tick_params(axis='both',labelsize=ticklabelsize)

ax2=fig.add_subplot(2,2,4)
ax2.semilogy(gammarange,redchisqrofgamma)
ax2.set_xlabel('$\\gamma$',fontsize=xlabelsize)
ax2.set_title('$\chi^2_r(\\gamma)$',fontsize=17)
ax2.tick_params(axis='both',labelsize=ticklabelsize)

plt.show()
####################################################################################